In [75]:
#import RapidML

from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import quandl
from datetime import datetime
API = ''
quandl.ApiConfig.api_key = API
tpot = TPOTRegressor(generations=5, population_size=50,verbosity=2, cv=TimeSeriesSplit(n_splits=15))


In [2]:
df = quandl.get("BSE/BOM532500")
print(df.head())
df = df[['Close']]
print(df.head())

              Open    High     Low   Close         WAP  No. of Shares  \
Date                                                                    
2003-07-09  158.40  170.25  156.10  164.05  165.601766     16127441.0   
2003-07-10  165.95  168.25  164.35  166.95  166.685458      3565356.0   
2003-07-11  167.50  174.80  166.25  173.40  172.271289      4566262.0   
2003-07-14  175.00  179.20  174.05  178.20  177.760520      2336576.0   
2003-07-15  179.80  181.80  173.00  176.35  176.785983      2368760.0   

            No. of Trades  Total Turnover  Deliverable Quantity  \
Date                                                              
2003-07-09        72917.0    2.670733e+09                   NaN   
2003-07-10        20210.0    5.942930e+08                   NaN   
2003-07-11        25626.0    7.866358e+08                   NaN   
2003-07-14        14128.0    4.153510e+08                   NaN   
2003-07-15        16230.0    4.187636e+08                   NaN   

            % Deli

In [3]:
df = df.reset_index()
print(df.head())

        Date   Close
0 2003-07-09  164.05
1 2003-07-10  166.95
2 2003-07-11  173.40
3 2003-07-14  178.20
4 2003-07-15  176.35


In [4]:
X = df.values[:,0]
ar = []
import time

for x in X:
    ar.append(time.mktime(x.timetuple()))
df['Date'] = ar

In [48]:
i = 0
ar2 = []
ar3 = []
iv = 24*60*60
s = ar[-1] 
for i in range(365*10):
    s+=iv
    ar2.append(s)
    ar3.append(datetime.fromtimestamp(s))


In [39]:
print(df.head())

           Date   Close
0  1.057680e+09  164.05
1  1.057766e+09  166.95
2  1.057853e+09  173.40
3  1.058112e+09  178.20
4  1.058198e+09  176.35


In [76]:
model = RapidML.rapid_regressor(df,model = tpot)


Using RapidML Regressor; Experimental, For Issues Contact Author: ritabratamaiti@hiretrex.com

Continuing without label encoding


Generation 1 - Current best internal CV score: -424886.65300039493


Generation 2 - Current best internal CV score: -391112.2071370531


Generation 3 - Current best internal CV score: -391112.2071370531


Generation 4 - Current best internal CV score: -386316.3705514855


Generation 5 - Current best internal CV score: -379455.88133692543



Best pipeline: GradientBoostingRegressor(input_matrix, alpha=0.85, learning_rate=1.0, loss=quantile, max_depth=10, max_features=0.8, min_samples_leaf=6, min_samples_split=9, n_estimators=100, subsample=0.7000000000000001)

Sample Output from input dataframe: 
1057852800.0,173.4


In [79]:
Y = []
Z = []


for x in df.values[:,0]:
    Y.append(model.predict(x))

for x in ar2:
    Z.append(model.predict(x))   

In [80]:


import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='', api_key='')
py.sign_in('', '')
    
data = [go.Scatter(x=X, y=df['Close'], name = "Actual"), go.Scatter(x=X, y=Y, name = "fitted"), go.Scatter(x = ar3, y=Z, name = "Predicted")]
py.iplot(data, filename = 'predictions_rapidML')